# Example Notebook:
This notebook provides an example on how to use the OOINet download tool to perform the following functions:
* Search for datasets
* Identify desired reference designator
* Get the associated metadata for a given reference designator
* Request netCDF datasets for a reference designator
* Download the netCDF dataset to your local machine

The key parameters which the OOINet API requires is the "reference designator." A reference designator may be thought of as a type of instrument located at a fixed location and depth. For example, below use the **CP01CNSM-RID27-03-CTDBPC000**, which is the CTD located at 7 meters depth on the Pioneer Array Central Surface Mooring at approximately (latitude, longitude) of (40.14, -70.7783).

In [1]:
# This is necessary if not installed as a package
import sys
sys.path.append("../ooinet/")

In [2]:
# Import the M2M module
from m2m import M2M

## Initialize the Tool
In order to utilize the OOINet tool, it needs to be initialized with your OOINet **username** and **token**. These may be found by logging onto ooinet.oceanobservatories.org and looking under your profile. If you have not registered with OOI, you cannot query OOINet via M2M, since it require authentication.

Personally, I store my OOI username and password locally in a yaml file which is excluded from git tracking. 

In [3]:
import yaml
import warnings
warnings.filterwarnings("ignore")
# Import user info for accessing UFrame
userinfo = yaml.load(open('../user_info.yaml'))
username = userinfo['apiname']
token = userinfo['apikey']

In [4]:
OOI = M2M(username, token)

In [5]:
OOI.username, OOI.token

('OOIAPI-C9OSZAQABG1H3U', 'JA48WUQVG7F')

---
## Search Datasets
First, we can search the available OOI Reference Designators (i.e. "refdes" for short) on the following keys: **array**, **node**, **instrument**. Additionally, can request for "**English_names**", which will return the descriptive name for the associated array, node, and instrument. Below, we will search for the available CTD instruments on the Pioneer Array Central Surface Mooring.

The major caveat with the search is, similar to searching on ERDDAP datasets, the search terms must be partial or full match based on OOI nomenclature. For example, we have to search for "CTD", "CTDBP", or the full instrument name "03-CTDBPC000". We can't search "conductivity", "temperature" or other CTD-related instrument terms.

In [6]:
datasets = OOI.search_datasets(array="CP01CNSM", instrument="CTD", English_names=True)
datasets

https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/CP01CNSM


,array,array_name,node,node_name,instrument,instrument_name,refdes,url,deployments
0,CP01CNSM,Coastal Pioneer Central Surface Mooring,RID27,Near Surface Instrument Frame,03-CTDBPC000,CTD,CP01CNSM-RID27-03-CTDBPC000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,CP01CNSM,Coastal Pioneer Central Surface Mooring,MFD37,Seafloor Multi-Function Node (MFN),03-CTDBPD000,CTD,CP01CNSM-MFD37-03-CTDBPD000,https://ooinet.oceanobservatories.org/api/m2m/...,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12]"


From the above datasets, we're going to select the CTDBP instrument on the Pioneer Array Central Surface Mooring Near-Surface Instrument Frame (located at 7m depth), which has a reference designator **CP01CNSM-RID27-03-CTDBPC000**. 

In [7]:
refdes = "CP01CNSM-RID27-03-CTDBPC000"

---
## Metadata
Next, we can query OOINet for the metadata associated with the selected reference designator. The metadata contains such valuable information such as the available methods and streams (which are required to download the data), the particleKeys (the data variable names), and the associated units. 

In [8]:
metadata = OOI.get_metadata(refdes=refdes)
metadata

,type,stream,units,fillValue,pdId,particleKey,shape,unsigned,method,count,endTime,beginTime,refdes
0,FLOAT,ctdbp_cdef_dcl_instrument,S m-1,-9999999,PD1,conductivity,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
1,FLOAT,ctdbp_cdef_dcl_instrument,dbar,-9999999,PD2,pressure,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
2,FLOAT,ctdbp_cdef_dcl_instrument,ºC,-9999999,PD6,temp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
3,DOUBLE,ctdbp_cdef_dcl_instrument,seconds since 1900-01-01,-9999999,PD7,time,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
4,DOUBLE,ctdbp_cdef_dcl_instrument,seconds since 1900-01-01,-9999999,PD10,port_timestamp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
5,DOUBLE,ctdbp_cdef_dcl_instrument,seconds since 1900-01-01,-9999999,PD11,driver_timestamp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
6,DOUBLE,ctdbp_cdef_dcl_instrument,seconds since 1900-01-01,-9999999,PD12,internal_timestamp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
7,STRING,ctdbp_cdef_dcl_instrument,1,empty,PD16,preferred_timestamp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
8,STRING,ctdbp_cdef_dcl_instrument,1,empty,PD93,date_time_string,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000
9,DOUBLE,ctdbp_cdef_dcl_instrument,seconds since 1900-01-01,-9999,PD863,ingestion_timestamp,SCALAR,False,telemetered,1864734,2020-08-04T18:03:08.334Z,2013-11-21T18:16:00.000Z,CP01CNSM-RID27-03-CTDBPC000


---
## Deployment Information
When we searched for CTD datasets on the Pioneer Central Surface Mooring, it returned a table which listed the available deployment numbers for each of the datasets. We can get much more detailed information on the deployments for a particular reference designator by requesting the deployment information from OOINet.

In [9]:
deployments = OOI.get_deployments(refdes=refdes)
deployments

,deploymentNumber,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,12,40.1332,-70.7783,7,2019-09-27 18:30:00,None,AR-39,None
0,11,40.1401,-70.7714,7,2019-04-06 14:35:00,2019-09-26 17:15:00,AR-34,AR-39
0,10,40.1334,-70.7777,7,2018-10-30 01:48:00,2019-04-07 18:08:00,AR-31,AR-34
0,9,40.1397,-70.7713,7,2018-03-24 21:32:00,2018-10-29 12:31:00,AR-28,AR-31
0,8,40.1334,-70.7783,7,2017-10-29 14:15:00,2018-03-29 19:37:00,AR-24,AR-28
0,7,40.1398,-70.7712,7,2017-06-09 14:24:00,2017-11-01 20:33:00,AR-18,AR-24
0,6,40.1334,-70.7785,7,2016-10-13 18:36:00,2017-06-09 16:05:00,AR-08,AR-18
0,5,40.1404,-70.7713,7,2016-05-13 13:50:00,2016-10-13 19:34:00,AR-04,AR-08
0,4,40.1332,-70.7784,7,2015-10-23 18:49:00,2016-04-04 12:03:00,AT-31,AR1-07
0,3,40.1402,-70.7713,7,2015-05-07 17:34:00,2015-10-23 19:40:00,AT-27,AT-31


---
## Vocab Information
Additionally, if we are interested in more detailed information on the location that the reference designator is assigned to, we can request the vocab information for the given reference designator. The vocab information includes some of the "**English_names**" info we requested when searching for datasets, as well as instrument model, manufacturer, and the descriptive names for the reference designator location.

In [10]:
vocab = OOI.get_vocab(refdes=refdes)
vocab

,@class,model,vocabId,refdes,instrument,tocL1,tocL2,tocL3,mindepth,maxdepth,manufacturer
0,.VocabRecord,SBE 16plusV2,472,CP01CNSM-RID27-03-CTDBPC000,CTD,Coastal Pioneer,Central Surface Mooring,Near Surface Instrument Frame,7.0,7.0,Sea-Bird


---
## Download Datasets
The ultimate goal of the queries above were to identify what data streams(s) we are interested in requesting data from to download. Now we want to be able to request those data streams and get the associated netCDF files. This process involves the following steps:
1. Identify the methods and data streams for the selected reference designator
2. Request the THREDDS server url for the data sets
3. Get the catalog of datasets on the THREDDS server
4. Parse the catalog for the desired netCDF files
5. Download the identified netCDF files to a local directory

**1.** Get the methods and data streams associated with the given reference designator:

In [11]:
streams = OOI.get_datastreams(refdes)
streams

,refdes,method,stream
0,CP01CNSM-RID27-03-CTDBPC000,recovered_host,ctdbp_cdef_dcl_instrument_recovered
1,CP01CNSM-RID27-03-CTDBPC000,recovered_inst,ctdbp_cdef_instrument_recovered
2,CP01CNSM-RID27-03-CTDBPC000,telemetered,ctdbp_cdef_dcl_instrument


**2.** Now, we request the THREDDS server url from OOINet. At a minimum, this requires the reference designator, method, and stream as inputs. This will request the datasets for _all_ deployments.

If we want to further limit the request to a specific deployment or a specific time period, we can do that by passing the arguments **beginDT** (begin datetime) and **endDT** (end datetime). 

Additionally, we can input some optional arguments that will return diagnostic information. The **include_provenance** will return a separate text file with information on the provenance of the data, such as the calibration coefficients applied. The **include_annotations** returns a separate text file of annotations, which are descriptions of issues and information associated with the given dataset.

In [12]:
method = "recovered_inst"
stream = "ctdbp_cdef_instrument_recovered"

In [13]:
thredds_url = OOI.get_thredds_url(refdes, method, stream)
thredds_url

'https://opendap.oceanobservatories.org/thredds/catalog/ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/catalog.html'

**3.** With the appropriate THREDDS url, we can query the THREDDS catalog to identify:

In [14]:
catalog = OOI.get_thredds_catalog(thredds_url)
catalog

['',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/status.txt',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/status.json',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0011_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_aggregate_provenance.json',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0011_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_20190406T144517-20190926T161808.nc',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0011_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered.ncml',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01C

**4.** Next, we want to parse the THREDDS catalog for the netCDF datasets which contain the relevant data. We can pass to the **exclude** keyword netCDF datasets which may be delivered as part of the THREDDS request, but which we aren't interested in. For example, Glider datasets frequently returned engineering (ENG) datastreams which contain engineering files but no relevant data.

In [15]:
netCDF_datasets = OOI.parse_catalog(catalog, exclude=["ENG", "gps"])
netCDF_datasets

['ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0011_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_20190406T144517-20190926T161808.nc',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0008_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_20171029T141519-20180329T190320.nc',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0007_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_20170609T142931-20171101T202931.nc',
 'ooi/areed@whoi.edu/20200804T213650353Z-CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered/deployment0006_CP01CNSM-RID27-03-CTDBPC000-recovered_inst-ctdbp_cdef_instrument_recovered_20161013T183731-20161122T190001.nc',
 'ooi/areed@whoi.edu/2020080

**5.** Finally, we can download the netCDF files to a specified save directory (**save_dir**):

In [17]:
save_dir = "/home/andrew/Documents/OOI-CGSN/ooinet/examples/"
OOI.download_netCDF_files(netCDF_datasets, save_dir=save_dir)

## Open netCDF Datasets
Instead of downloading netCDF datasets to a local directory, another option is to load the netCDF datasets right from the THREDDS OpenDAP server into an xarray dataset.

In [18]:
ctd = OOI.load_netCDF_files(netCDF_datasets)

In [19]:
ctd

,Array,Chunk
Bytes,9.57 MB,2.43 MB
Shape,"(1196042,)","(303449,)"
Count,670 Tasks,649 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,4.78 MB,1.21 MB
Shape,"(1196042,)","(303449,)"
Count,670 Tasks,649 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
